<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries


# Explore some ocean data, load an external csv file and do some filtering. Make some quick and dirty maps using plotly express

Also demonstrating the use of pickle for storing intermediate results

Loading required packages.
If PyCO2SYS is not installed, then uncomment the first line for the first run.

In [ ]:
#!pip install PyCO2SYS
import PyCO2SYS as pyco2
import pandas as pd
from icoscp_core.icos import bootstrap,auth, meta,data, OCEAN_STATION
import plotly.express as px
import os.path

def humanbytes(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2) 
    GB = float(KB ** 3) 
    TB = float(KB ** 4) 
    if B < KB:
        return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
    elif KB <= B < MB:
        return '{0:.2f} KB'.format(B / KB)
    elif MB <= B < GB:
        return '{0:.2f} MB'.format(B / MB)
    elif GB <= B < TB:
        return '{0:.2f} GB'.format(B / GB)
    elif TB <= B:
        return '{0:.2f} TB'.format(B / TB)

In [ ]:
#auth.init_config_file()  # only for the first run, then move .password to your root directory with attrib 400
auth,meta,data = bootstrap.fromPasswordFile('~/.password')

In [ ]:
all_datatypes = meta.list_datatypes()
previewable_datatypes = [dt for dt in all_datatypes if ((dt.has_data_access) & (dt.theme.label=='Ocean data') ) ]
previewable_datatypes

In [ ]:
oceanStations = meta.list_stations(OCEAN_STATION)
station = [st for st in oceanStations if ('Tukuma' in st.label) ]
station

In [ ]:
otcReleaseType='http://meta.icos-cp.eu/resources/cpmeta/icosOtcL2Product'
tukumaArctica='http://meta.icos-cp.eu/resources/stations/OS_26RA'
latest_release = meta.list_data_objects(datatype=otcReleaseType,station=tukumaArctica)
latest_release[0]

In [ ]:
dobj_meta = meta.get_dobj_meta(latest_release[0].uri)
dobj_meta

In [ ]:
dobj_arrays = data.get_columns_as_arrays(dobj_meta)
dobj_arrays

In [ ]:
dobj_arrays = data.get_columns_as_arrays(dobj_meta,['TIMESTAMP','Latitude','Longitude','fCO2 [uatm]'])
df = pd.DataFrame(dobj_arrays)
display(df)

In [ ]:
df = df[df['fCO2 [uatm]'] > 0]
fig = px.scatter_geo(df,lat='Latitude',lon='Longitude', 
                     fitbounds='locations',
                     hover_data='fCO2 [uatm]', 
                     hover_name = 'TIMESTAMP',
                     color='fCO2 [uatm]', range_color=[330,410], 
                     width = 1024, height=500,
                     projection= 'natural earth')
fig.update_geos(resolution=50,
                showcoastlines=True, coastlinecolor="RebeccaPurple",
                showland=True, landcolor="LightGreen",
                showocean=True, oceancolor="LightBlue",
                showlakes=True, lakecolor="Blue")
fig.update_layout(title = station[0].label+' '+dobj_meta.fileName, title_x=0.5)
fig.show()

In [ ]:
if not(os.path.isfile('data/glodap.pkl')):
    glodap_df = pd.read_csv('data/GLODAPv2.2023_Arctic_Ocean.zip',sep=',',decimal='.',compression='zip')
    glodap_df.rename(columns={'G2year':'year','G2month':'month','G2day':'day','G2hour':'hour','G2minute':'minute'},inplace=True)
    glodap_df['year']=glodap_df['year'].astype('Int16') 
    glodap_df['month']=glodap_df['month'].astype('Int16') 
    glodap_df['day']=glodap_df['day'].astype('Int16') 
    glodap_df['hour']=glodap_df['hour'].astype('Int16') 
    glodap_df['minute']=glodap_df['minute'].astype('Int16') 
    glodap_df['Datetime'] = pd.to_datetime(glodap_df[['year','month','day','hour','minute']])
    glodap_df.drop(['year','month','day','hour','minute'],axis=1,inplace=True)
    glodap_df.set_index('Datetime',inplace=True)
    glodap_df.to_pickle('data/glodap.pkl') # now save for next time

In [ ]:
glodap_df=pd.DataFrame()
glodap_df=pd.read_pickle('data/glodap.pkl')
print('Memory size of glodap dataframe: ', humanbytes(glodap_df.memory_usage().sum()) )
glodap_df

In [ ]:
selected=glodap_df[(glodap_df['G2depth'] <10) 
                   & (glodap_df['G2fco2']>0) 
                   & (glodap_df.index.year>2014)].filter(['G2fco2','G2latitude','G2longitude'],axis=1)
fig = px.scatter_geo(selected,lat='G2latitude',lon='G2longitude', 
                     fitbounds='locations',
                     hover_data = 'G2fco2', 
                     color='G2fco2', 
                     width = 1024, height=500,
                     projection= 'natural earth')
fig.update_geos(resolution=50,showcoastlines=True,
                coastlinecolor="RebeccaPurple",
                showland=True,landcolor="LightGreen",
                showocean=True,oceancolor="LightBlue",
                showlakes=True,lakecolor="Blue")
fig.update_layout(title = 'GLODAP 2015-2021', title_x=0.5)
fig.show()